In [1]:
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib.patches import Circle, Rectangle
from matplotlib.collections import PatchCollection
import numpy as np
from scipy import signal
from scipy import ndimage

%matplotlib

np.random.seed(77777)

Using matplotlib backend: Qt5Agg


In [2]:
def gen_template(w: float, h: float, dpi: int, loc: str, plot=False):
    '''
    Generate an image of a photogrammetry target.

    Parameters
    ----------
    w
        total image width, m
    h
        total image height, m
    dpi
        output image dpi, converts between matrix coordinates and real width
    loc
        [SW, NW, SE, NE, raft] generates a different pattern for each loc
    '''
    IN_TO_METERS = 0.0254
    w_in = w / IN_TO_METERS
    h_in = h / IN_TO_METERS
    w_px = np.round(w_in * dpi).astype(int)
    h_px = np.round(h_in * dpi).astype(int)

    fig, ax = plt.subplots()
    fig.set_size_inches(w_in, h_in)
    fig.tight_layout()
    fig.subplots_adjust(left=0., right=1., top=1., bottom=0.)
    ax.scatter([0, 0, w_px, w_px], [0, h_px, 0, h_px], color='k')

    # black background
    # ax.add_patch(Rectangle((0, 0), w_px, h_px, color='k'))
    pattern_size = 8
    pattern = np.random.random((pattern_size,pattern_size)) * 0.5 + .5
    pattern[0][0] = 0
    pattern[pattern_size - 1][pattern_size - 1] = 1
    ax.imshow(ndimage.zoom(pattern, h_px / pattern_size, order=0), cmap='Greys')

    # center cross
    c_size = dpi * 0.01 / IN_TO_METERS # 10 cm
    c_thick = dpi * 0.001 / IN_TO_METERS # 1 mm
    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 2), c_size, c_thick, color='w')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 2, h_px / 2 - c_size / 2), c_thick, c_size, color='w')) # vert

    circ_diam = c_size
    if loc == 'SW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4, h_px / 4),
        )
    elif loc == 'NW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'NE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'SE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, h_px / 4),
        )
    elif loc == 'raft':
        locs = (
            (w_px / 4, h_px / 4),
            (w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, h_px / 4),
        )
    else:
        raise ValueError(f'loc {loc} is not an option')

    for elem in locs:
        ax.add_patch(Circle(elem, circ_diam / 2, color='w', zorder=1))

    ax.set_aspect('equal')
    ax.set_xlim([0, w_px])
    ax.set_ylim([0, h_px])
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(f'{loc}_target.png')
    if not plot:
        plt.close()

plt.close('all')
locs = ['NW', 'SW', 'NE', 'SE', 'raft']
# for loc in locs:
    # gen_template(0.04, 0.04, 300, loc, plot=False)

In [3]:
f0 = plt.figure(constrained_layout=True)
axd = f0.subplot_mosaic(
    """
    aa
    bc
    """
)
im = image.imread('2022-01-09 22.37.05.jpeg')
im = np.sum(im.astype(float), axis=2) # single channel
im = np.max(im) - im # invert value
im -= im.mean() # detrend
stride = 3
im = im[100:-500:stride,500:-500:stride]
axd['a'].imshow(im, cmap='Greys')


In [4]:
# plot convergence
f1, ax = plt.subplots(nrows=5)
f1.tight_layout()

for i, loc in enumerate(locs):
    ax[i].set_title(loc)
    ax[i].set_xlabel('Template Image Scale Factor')
    ax[i].set_ylabel('Peak Correlation Value')
    l = None

    maxes = []
    tmp = image.imread(loc + '_target.png')
    tmp = np.max(tmp) - tmp # invert
    tmp = np.sum(tmp.astype(float) , axis=2) # single channel
    tmp -= tmp.mean() # detrend
    # try different scales
    sz = np.max(im.shape)
    zooms = np.linspace(0.4 * sz, 1.5 * sz, num=50) / np.max(im.shape)
    for j, zoom in enumerate(zooms):
        # print(f'Trying zoom {100. * zoom} % ...')
        tmp_zoom = ndimage.zoom(tmp, zoom) # apply zoom
        xcorr = signal.fftconvolve(im, tmp_zoom[::-1, ::-1], mode='valid')
        max_val = np.max(xcorr)
        y,x = np.unravel_index(np.argmax(xcorr), xcorr.shape)
        maxes.append(max_val)

        axd['b'].imshow(xcorr, cmap='Greys')
        axd['c'].imshow(tmp_zoom, cmap='Greys')
        if not l:
            l, = ax[i].plot(zooms[:len(maxes)], maxes)
        else:
            l.set_data(zooms[:len(maxes)], maxes)
            ax[i].set_xlim(.9 * min(zooms[:len(maxes)]), max(zooms[:len(maxes)]) * 1.1)
            ax[i].set_ylim(.9 * min(maxes), max(maxes) * 1.1)

        # Generally, a match has been found when the scale reaches its first maximum.
        # It will then increase again without bound, so select the first max only.
        if (len(maxes) > 2) and (maxes[-1] < maxes[-2]):
            print(f'{loc} target is at {(x, y)} in image')
            axd['a'].add_patch(Rectangle((x, y), tmp_zoom.shape[0], tmp_zoom.shape[1], fill = False, color='red'))
            axd['a'].scatter(x,y)
            axd['a'].annotate(f'{loc}, zoom={zoom:.2f}', (x,y))
            break
        
        f0.canvas.draw_idle()
        f1.canvas.draw_idle()
        plt.pause(0.001)



NW target is at (52, 97) in image
SW target is at (66, 510) in image
NE target is at (725, 132) in image
SE target is at (696, 497) in image
raft target is at (374, 286) in image
